In [23]:
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Funkcija, kas ielādē datus
def load_data():
    # ratings.csv ielāde
    ratings_df = pd.read_csv(r"C:\Bakalaurs_praktiskais\Bakalaura-darbs\ratings.csv")
    # Pārbauda kolonas
    if 'userId' in ratings_df.columns:
        ratings_df = ratings_df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
    
    # movies.csv ielāde
    movies_df = pd.read_csv(r"C:\Bakalaurs_praktiskais\Bakalaura-darbs\movies.csv")
    if 'movieId' in movies_df.columns:
        movies_df = movies_df.rename(columns={'movieId': 'movie_id'})
    
    # Žanru apstrāde priekš TF-IDF
    movies_df['genres'] = movies_df['genres'].apply(lambda x: x.replace('|', ' '))
    
    print(f"Loaded data: {len(ratings_df)} ratings and {len(movies_df)} movies")
    return ratings_df, movies_df

In [ ]:
# Matricu faktorizācijas modelis no Suprise bibliotēkas
class SVDModel:
    def __init__(self, n_factors=35, n_epochs=25, lr_all=0.008, reg_all=0.08):
        self.model = SVD(n_factors=n_factors, n_epochs=n_epochs, 
                         lr_all=lr_all, reg_all=reg_all)
        self.trainset = None
        self.testset = None
    
    def prepare_data(self, ratings_df):
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
        self.trainset, self.testset = train_test_split(data, test_size=0.2, random_state=42)
        return self.trainset, self.testset
    
    def train(self, trainset=None):
        if trainset is not None:
            self.trainset = trainset
        self.model.fit(self.trainset)
    
    def evaluate(self, testset=None):
        if testset is not None:
            self.testset = testset
        predictions = self.model.test(self.testset)
        return predictions
    
    def cross_validate(self, data, cv=5):
        return cross_validate(self.model, data, measures=['RMSE', 'MAE'], cv=cv, verbose=True)
    
    def predict_rating(self, user_id, item_id):
        return self.model.predict(user_id, item_id).est

In [ ]:
# TF-IDF modelis
class TFIDFModel:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = None
        self.movies_df = None
        self.cosine_sim = None
    
    def fit(self, movies_df):
        self.movies_df = movies_df
        self.tfidf_matrix = self.vectorizer.fit_transform(movies_df['genres'])
        self.cosine_sim = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)
        return self.cosine_sim
    
    def get_recommendations(self, movie_id, top_n=10):
        movie_idx = self.movies_df[self.movies_df['movie_id'] == movie_id].index[0]
        sim_scores = list(enumerate(self.cosine_sim[movie_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]  # Exclude the movie itself
        movie_indices = [i[0] for i in sim_scores]
        return self.movies_df.iloc[movie_indices]

**Jaunais eval**

In [ ]:
class HybridRecommender:
    def __init__(self, svd_weight=0.5):
        self.svd_model = SVDModel()
        self.tfidf_model = TFIDFModel()
        self.svd_weight = svd_weight
        self.ratings_df = None
        self.movies_df = None

    def train(self, ratings_df, movies_df):
        #Apmāca modeļus
        self.ratings_df = ratings_df
        self.movies_df = movies_df

        # Apmāca Matricu faktorizācijas modeli
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
        self.svd_model.prepare_data(ratings_df)
        self.svd_model.train()

        # Apmāca TF-IDF modeli
        self.tfidf_model.fit(movies_df)

        return self

    def get_movie_prediction(self, user_id, movie_id, user_high_rated=None, min_rating=3.5):
        
        # Iegūst MF prognozes
        svd_pred = self.svd_model.predict_rating(user_id, movie_id)
        
        # Iegūst TF-IDF prognozes
        if movie_id in self.movies_df['movie_id'].values:
            movie_idx = self.movies_df[self.movies_df['movie_id'] == movie_id].index[0]
            cb_scores = self.tfidf_model.cosine_sim[movie_idx]
            
            
            if user_high_rated is None:
                user_high_rated = self.ratings_df[
                    (self.ratings_df['user_id'] == user_id) &
                    (self.ratings_df['rating'] >= min_rating)
                ]
            
            # Aprēķina CB vērtējumu
            if not user_high_rated.empty:
                cb_sum = 0
                count = 0
                
                for _, row in user_high_rated.iterrows():
                    rated_movie_id = row['item_id']
                    if rated_movie_id in self.movies_df['movie_id'].values:
                        rated_idx = self.movies_df[
                            self.movies_df['movie_id'] == rated_movie_id
                        ].index[0]
                        cb_sum += cb_scores[rated_idx]
                        count += 1
                
                # Aprēķina gala CB vērtējumu
                cb_score = cb_sum / count if count > 0 else 0
                
                # Aprēķina hibrīdo vērtējumu
                hybrid_pred = (
                    self.svd_weight * svd_pred +
                    (1 - self.svd_weight) * cb_score
                )
                
                return hybrid_pred, svd_pred, cb_score
        
        # Ja CB nav prognozes, izmanto tikai matricu faktorizāciju
        return svd_pred, svd_pred, 0

    # Izveido ieteikumus lietotājam
    def recommend(self, user_id, top_n=10, min_rating=4, cf_candidates=300):
        """Generate hybrid recommendations for a given user."""
        # Filmas, kuras lietotājs nav vērtējis
        user_rated_movies = set(self.ratings_df[self.ratings_df['user_id'] == user_id]['item_id'])
        all_movies = set(self.movies_df['movie_id'])
        unrated_movies = list(all_movies - user_rated_movies)
        
        if not unrated_movies:
            return pd.DataFrame() # Ja nav filmas ko ieteikt, tad atgriež tukšu kopu
        
        # Iegūst filmas ar augstu vērtējumu
        user_high_rated = self.ratings_df[
            (self.ratings_df['user_id'] == user_id) &
            (self.ratings_df['rating'] >= min_rating)
        ]
        
        if user_high_rated.empty:
            # Ja nav, tad izmanto tikai matricu faktorizāciju
            svd_predictions = {
                movie_id: self.svd_model.predict_rating(user_id, movie_id)
                for movie_id in unrated_movies
            }
            # Sakārto filmas, kuras iesaka
            top_movies = sorted(svd_predictions.items(), key=lambda x: x[1], reverse=True)[:top_n]
            
            result = []
            for movie_id, pred in top_movies:
                if movie_id in self.movies_df['movie_id'].values:
                    movie_info = self.movies_df[self.movies_df['movie_id'] == movie_id].iloc[0]
                    result.append({
                        'movie_id': movie_id,
                        'title': movie_info['title'],
                        'hybrid_score': pred,
                        'cf_score': pred,
                        'cb_score': 0
                    })
            
            return pd.DataFrame(result)
        
        # Prognozē vērtējumus, izmantojot matricu faktorizāciju
        svd_predictions = {
            movie_id: self.svd_model.predict_rating(user_id, movie_id)
            for movie_id in unrated_movies
        }

        # Iegūst ieteikumus, ko sniedzis matricu faktorizācija
        cf_recommendations = sorted(
            svd_predictions.items(), key=lambda x: x[1], reverse=True
        )[:cf_candidates]  

        # Apvieno rezultātus hibrīdajā modelī
        hybrid_scores = []

        for movie_id, cf_score in cf_recommendations:
            hybrid_pred, svd_pred, cb_score = self.get_movie_prediction(
                user_id, movie_id, user_high_rated, min_rating
            )
            
            if hybrid_pred > 0:  
                hybrid_scores.append((movie_id, hybrid_pred, svd_pred, cb_score))

        # Atrgiež gala ieteikumu sarakstu
        recommendations = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:top_n]

        result = []
        for movie_id, hybrid_score, cf_score, cb_score in recommendations:
            movie_info = self.movies_df[self.movies_df['movie_id'] == movie_id].iloc[0]
            result.append({
                'movie_id': movie_id,
                'title': movie_info['title'],
                'hybrid_score': hybrid_score,
                'cf_score': cf_score,
                'cb_score': cb_score
            })

        return pd.DataFrame(result)

   # Funkcija, kas aprēķina novērtēšanas metrikas, precizitāti, atsaukumu, F1, MAE un RMSE
    def evaluate_hybrid(self, test_users=20, top_n=20, train_ratio=0.8, liked_threshold=4.0):
       
        # Filtrē lietotājus ar konkrētu novērtētu filmu sakitu (200)
        users_with_min_ratings = self.ratings_df['user_id'].value_counts()
        qualified_users = users_with_min_ratings[users_with_min_ratings >= 200].index.tolist()
        
        if len(qualified_users) == 0:
            return pd.DataFrame({"error": ["No users with enough ratings found"]})
            
        # Izvēlās lietotājus, kurus novērtēt uz nejaušibas principa
        test_users = np.random.choice(
            qualified_users,
            min(test_users, len(qualified_users)),
            replace=False
        )

        # Atgriež metrikas
        results = {
            'user_id': [],
            'num_ratings': [],
            'num_liked_test': [],
            'precision': [],
            'recall': [],
            'f1_score': [],
            'hit_rate': [],
            'mae': [],
            'rmse': []
        }

        for user_id in test_users:
            # Iegūst visus vērtējumus konkrētam lietotājam
            user_data = self.ratings_df[self.ratings_df['user_id'] == user_id]
            num_ratings = len(user_data)
            
            # Izlaiž lietotājus, kuriem ir pārāk maz vērtējumu par filmām
            if num_ratings < 200:
                continue
                
            # Apmācīšanas un testēšanas sadalījums
            train_data = user_data.sample(frac=train_ratio, random_state=42)
            test_data = user_data.drop(train_data.index)
            
            # Definē, kuras filmas ir derīgas (actual liked >=4)
            actual_liked = set(test_data[test_data['rating'] >= liked_threshold]['item_id'])
            num_liked_test = len(actual_liked)
            
            if num_liked_test == 0:
                continue  

            # Izveido modeli uz testēšanas datiem
            temp_ratings = pd.concat([
                self.ratings_df[self.ratings_df['user_id'] != user_id],
                train_data
            ])
            temp_hybrid = HybridRecommender(svd_weight=self.svd_weight)
            temp_hybrid.train(temp_ratings, self.movies_df)
            
            # Iegūst ieteikumus no recommend funkcijas
            hybrid_recs = temp_hybrid.recommend(
                user_id, 
                top_n=top_n,
                min_rating=liked_threshold,  d 
                cf_candidates=min(100, len(self.movies_df) - len(user_data))  
            )
            
            if hybrid_recs.empty:
                continue
                
            hybrid_recs_set = set(hybrid_recs['movie_id'])
            
            # Aprēķina precizitāte, atsaukumu un F1
            true_positives = len(hybrid_recs_set.intersection(actual_liked))
            precision = true_positives / len(hybrid_recs_set) if hybrid_recs_set else 0
            recall = true_positives / num_liked_test if num_liked_test > 0 else 0
            f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # Precīzi iegūtie ieteikumi
            hit_rate = 1 if true_positives > 0 else 0
            
            # Aprēķina MAE un RMSE
            mae_sum = 0
            rmse_sum = 0
            count = 0
            
            # Iegūst lietotājus ar daudz vērtējumiem, kas atbilst norādītajai robežvērtībai
            user_high_rated = train_data[train_data['rating'] >= liked_threshold]

            for _, row in test_data.iterrows():
                movie_id = row['item_id']
                actual_rating = row['rating']
                
                
                hybrid_pred, _, _ = temp_hybrid.get_movie_prediction(
                    user_id, movie_id, user_high_rated, min_rating=liked_threshold
                )
                
                if hybrid_pred > 0:  
                    error = abs(actual_rating - hybrid_pred)
                    squared_error = error ** 2
                    mae_sum += error
                    rmse_sum += squared_error
                    count += 1
            
            # Aprēķina gala MAE un RMSE
            mae = mae_sum / count if count > 0 else float('inf')
            rmse = np.sqrt(rmse_sum / count) if count > 0 else float('inf')
            
            # Saglabā rezultātus
            results['user_id'].append(user_id)
            results['num_ratings'].append(num_ratings)
            results['num_liked_test'].append(num_liked_test)
            results['precision'].append(precision)
            results['recall'].append(recall)
            results['f1_score'].append(f1)
            results['hit_rate'].append(hit_rate)
            results['mae'].append(mae)
            results['rmse'].append(rmse)
        
        return pd.DataFrame(results)

**Jaunais main**

In [ ]:
def main():
    # Ielādē datukopas ratings.csv un movies.csv
    ratings_df, movies_df = load_data()
   
    # Apmāca matricu faktorizācijas modeli
    svd_model = SVDModel()
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
    cv_results = svd_model.cross_validate(data)
    print(f"SVD Cross-validation results: {cv_results}")
   
    # Apmāca Hibrīdo modeli
    hybrid_model = HybridRecommender(svd_weight=0.7)
    hybrid_model.train(ratings_df, movies_df)
   
    # Novērtē hibrīdo modeli
    eval_results = hybrid_model.evaluate_hybrid(test_users=20, top_n=10)
   
    # Izdrukā novērtēšanas metrikas
    print("\nHybrīdā modeļa metrikas")
    print(eval_results.describe())
    
    # Izprintē vidējās novērtēšanas metrikas hibrīdajam modelim
    print("\nVidējās hibrīdā modeļa metrikas:")
    print(f"Precizitāte: {eval_results['precision'].mean():.4f}")
    print(f"Atsaukums: {eval_results['recall'].mean():.4f}")
    print(f"F1: {eval_results['f1_score'].mean():.4f}")
    print(f"MAE: {eval_results['mae'].mean():.4f}")
    print(f"RMSE: {eval_results['rmse'].mean():.4f}")
    
    # Veido ieteikumus lietotājam
    test_user_id = ratings_df['user_id'].sample(1).iloc[0]
    print(f"\nIeteikumu saraksts lietotājam {test_user_id}:")
    
    recommendations = hybrid_model.recommend(test_user_id, top_n=5)
    
    print("\nTop 5 ieteikumi:")
    for i, (_, row) in enumerate(recommendations.iterrows(), 1):
        print(f"{i}. {row['title']} (Hybrid Score: {row['hybrid_score']:.4f})")

if __name__ == "__main__":
    main()

Loaded data: 100836 ratings and 9742 movies
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8656  0.8752  0.8789  0.8740  0.8792  0.8746  0.0049  
MAE (testset)     0.6681  0.6728  0.6731  0.6692  0.6744  0.6715  0.0024  
Fit time          1.32    1.37    1.25    1.26    1.23    1.28    0.05    
Test time         0.18    0.11    0.11    0.10    0.15    0.13    0.03    
SVD Cross-validation results: {'test_rmse': array([0.86562031, 0.87521147, 0.87893926, 0.87403783, 0.87918874]), 'test_mae': array([0.66811248, 0.67284036, 0.67309632, 0.66921133, 0.67437886]), 'fit_time': (1.3163225650787354, 1.3661184310913086, 1.250467300415039, 1.2590129375457764, 1.2250723838806152), 'test_time': (0.18290257453918457, 0.11418366432189941, 0.10774111747741699, 0.09986734390258789, 0.14893507957458496)}
Training hybrid recommender...
Evaluating hybrid recommender...

Hybrid Recommender Evaluation Resul